In [1]:
import requests
import urllib.request
import time as time
from bs4 import BeautifulSoup, SoupStrainer
from tkinter import *
# from PIL import Image, ImageTk
from io import BytesIO
import pandas as pd
import os

import dash
import lxml
from time import sleep
from random import randint
import numpy as np

In [49]:
class Soup :

    def __init__(self, url, *args):
        self.url = url
        self.soup = self.urlToSoup(url)
        if args :
            self.soup = self.urlToSoup(url, args[0])


    def urlToSoup(self, url, *args):
        html_content = requests.get(url=url).text
        if args :
            return BeautifulSoup(html_content, 'lxml', parse_only=args[0])
        return BeautifulSoup(html_content , 'lxml')

    def getAllElement(self, type, *args):
        l = self.soup.find_all(type)
        liste = list()
        for e in l:
            print(e)
            if 'class' in e.attrs.keys() :
                if len(e.attrs['class']) == 2 :
                    if ''.join(e.attrs['class'][1]) == args[0] :
                        liste.append(e)
        return liste

    def getElements(self, type, *args):
        l = self.soup.find_all(type)
        liste = list()
        for e in l:
            if 'class' in e.attrs.keys() :
                if len(e.attrs['class']) > 0 :
                    if e.attrs['class'][0] == args[0] :
                        liste.append(e)
        return liste

    def showHyperlinks(self):
        for link in self.soup.find_all('a'):
            url = link.get('href')
            if url[:5] == "https" or url[:4] == "http":
                print(link.get('href'))
            else:
                print(f"https://{url}")

    def __str__(self):
        return f'URL : {self.url}'

In [3]:
pageUrls = ["https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza"]
pageUrls += ["https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=" + str(i) for i in range(2, 81)]
pageUrls

['https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=2',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=3',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=4',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=5',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=6',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=7',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=8',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=9',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=10',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=11',
 'https://www.immobiliare.it/en/affi

In [52]:
SOUP_RENT = list()
%time SOUP_RENT = [Soup(url) for url in pageUrls]
# SOUP_SALE = [Soup("https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza")]
# %time SOUP_SALE += [Soup("https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=" + str(i)) for i in range(2, 81)]
# appartments = list()
# for i in range(len(SOUP_RENT)):
#     appartments += [appart.div.a['href'] for appart in SOUP_RENT[i].getAllElement('li', 'in-realEstateResults__item')]

CPU times: total: 5.34 s
Wall time: 1min 8s
CPU times: total: 4.42 s
Wall time: 24.6 s


In [55]:
soup1 = SOUP_RENT[0]
soup1.getAllElement('div', 'in-reListCard__content')

<div id="__next"><div class="in-listAndMapHeader"><header class="nd-navbar in-navbar"><nav class="nd-navbar__row"><section class="nd-navbar__section nd-navbar__section--brand nd-navbar__section--brandWithSearch"><a aria-label="immobiliare.it" class="nd-navbar__link in-header__logo" href="https://www.immobiliare.it/en/"><picture class="nd-navbar__logo"><source height="30" media="(max-width: 1023px)" srcset="https://s1.immobiliare.it/_next/static/media/logo-immobiliare-mark.804588ee856470b8.svg" width="26"/><img alt="immobiliare.it" src="https://s1.immobiliare.it/_next/static/media/logo-immobiliare-alt.99b47712664189ed.svg"/></picture></a></section><section class="nd-navbar__section nd-navbar__section--search"><div class="nd-navbar__item nd-navbar__item--search" role="button"><nd-geography-search autorefill="true" class="nd-autocomplete im-geographySearch" countrysearchenabled="true" drawenabled="true" internationalenabled="true" internationallabelsenabled="true" metroenabled="true" moda

[]

In [10]:
print(len(SOUP_RENT))

80


In [5]:
print(len(appartments))

0


In [8]:
estateFeaturesName = list()
estateFeaturesValue = list()
cities = list()
districts = list()
streets = list()

for a in appartments :
    apartment = Soup(a)
    adressElements = apartment.getElements('span', 'in-location')
    try:
        city = adressElements[0].contents[0]
    except IndexError :
        city = "NA"
    try:
        district = adressElements[1].contents[0]
    except IndexError :
        district = "NA"
    try:
        street = adressElements[2].contents[0]
    except IndexError :
        street = "NA"
    cities.append(city)
    districts.append(district)
    streets.append(street)
    estateFeaturesName.append(apartment.getElements('dt', 'in-realEstateFeatures__title'))
    estateFeaturesValue.append(apartment.getElements('dd', 'in-realEstateFeatures__value'))

dataDict = dict()
n = len(estateFeaturesValue)
for i in range(n):
    # i = appartment index
    m = len(estateFeaturesValue[i])
    for j in range(m):
        key = estateFeaturesName[i][j].contents[0]
        if key not in dataDict.keys():
            dataDict[key] = list()
for i in range(n):
    m = len(estateFeaturesValue[i])
    for j in range(m):
        key = estateFeaturesName[i][j].contents[0]
        val = estateFeaturesValue[i][j].contents[0]
        if val != 'Open to participate':
            dataDict[key].append(val)
    for k, v in dataDict.items():
        if len(v) == i:
            dataDict[k].append("NA")

dataDict["city"] = cities
dataDict["district"] = districts
dataDict["street"] = streets

{'city': [], 'district': [], 'street': []}


In [7]:
dataframe = pd.DataFrame(dataDict)

# Add column of urls
def make_clickable(url) :
    return f'<a href="{url}">{url}</a>'

clickableLinks = [make_clickable(url) for url in appartments]
dataframe['URL'] = appartments
newDataframe = dataframe.style.format( {'URL' : make_clickable} )

# Edit prices column
dataframe = dataframe[dataframe['price']!="NA"]
dataframe = dataframe[dataframe['surface']!="NA"]
dataframe = dataframe[dataframe['price']!="price on application"]
dataframe['price']= dataframe['price'].str.replace("€ ","")
dataframe['price']= dataframe['price'].str.replace("/month","")
dataframe['price']= dataframe['price'].str.replace(",","").astype(float)
dataframe = dataframe.sort_values('price')
dataframe = dataframe.rename(columns={"price": "price (k€)"})

# Edit surfaces column
dataframe['surface'] = dataframe['surface'].str.replace(" m²","")
dataframe['surface'] = dataframe['surface'].str.extract(r'^(\d+)').astype(int)

# Edit floors column
dataframe['floor'] = dataframe['floor'].str.replace(r'^Ground floor.*$', '0', regex=True)
dataframe['floor'] = dataframe['floor'].str.replace(r'^Mezzanine.*$', '1', regex=True)
dataframe['floor'] = dataframe['floor'].str.replace(r'°.*$', "", regex=True)
dataframe['district'] = dataframe['district'].str.replace(r"'", '', regex=True)
dataframe['street'] = dataframe['street'].str.replace(r"'", '', regex=True)

KeyError: 'price'

In [ ]:
newDataframe
# dataframe[['city', 'district', 'street' ,'surface', 'price (k€)', 'rooms', 'floor', 'condition', 'contract', 'URL']]

In [ ]:
dataframe.to_pickle('rental.pkl')
dataframe[['city', 'district', 'street' ,'surface', 'price (k€)', 'rooms', 'floor', 'condition', 'contract', 'URL']]